# Quantum Support Vector Machines

1. **Feature Map** for data encoding2. **#Quantum Kernel** for information processing
3. **Measurement** to get the outcome 

The quantum kernel is created by mapping a classical feature vector $x$ to a Hilbert space using a quantum feature map $|\phi(x)\rangle$. 

Mathematically:
$$K_{ij} = |\langle\phi(\vec{x_i})|\phi(\vec{x_j})\rangle|^2$$

where 
- $K_{ij}$ is the kernel matrix 
- $x_i$ and $x_j$ are n-dimensional inputs
- $|\phi(x)\rangle$ is the quantum feature map
- $|\langle a|b\rangle|^2$ denotes the overlap of two quantum states $a$ and $b$.

<!-- 
Source:
https://www.youtube.com/watch?v=5Kr31IFwJiI
https://medium.com/@roysuman088/unveiling-the-power-of-quantum-kernel-based-machine-learning-with-qiskit-8436b9ba41fb
-->

In [2]:
import pennylane as qml
from pennylane import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

#Load Data
X, y = load_iris(return_X_y=True)

# pick inputs and labels from the first two classes only, corresponding to the first 100 samples
X = X[:100]
y = y[:100]

# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# scaling the labels to -1, 1 is important for the SVM and the definition of a hinge loss
y_scaled = 2 * (y - 0.5)

# Split data in train and test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled)

n_qubits = len(X_train[0])
dev_kernel = qml.device("lightning.qubit", wires=n_qubits)

projector = np.zeros((2**n_qubits, 2**n_qubits))
projector[0, 0] = 1

# Generate Kernel for single element in quantum
@qml.qnode(dev_kernel, interface="autograd")
def kernel(x1, x2):
    """The quantum kernel."""
    qml.AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(qml.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

# Generate whole Kernel Matrix
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""
    return np.array([[kernel(a, b) for b in B] for a in A])

In [3]:
q_svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)

In [4]:
#Score
train_score_q4 = q_svm.score(X_train, y_train)
test_score_q4 = q_svm.score(X_test, y_test)

print(f"Train Score: {train_score_q4:.2f},  Test Score: {test_score_q4:.2f}")

Train Score: 1.00,  Test Score: 1.00
